TODO: change to use own model instead of pretrained

In [87]:
from pathlib import Path
import yaml
from pyannote.audio import Pipeline, Model
import os
from dotenv import load_dotenv
import re
from pydub import AudioSegment
from pyannote.audio.pipelines import SpeakerDiarization

load_dotenv()

huggingface_token = os.environ.get("HUGGINGFACE_TOKEN")

In [84]:
pipeline = Pipeline.from_pretrained(Path("./pipeline/config.yaml"))
model = Model.from_pretrained("./models/trained_model.ckpt")
pipeline.segmentation_model = model
pipeline.klustering = "OracleClustering"

with open("./parameters/config.yaml", "r") as f:
    parameters = yaml.safe_load(f)

params = parameters["params"]

pipeline.freeze({
        "segmentation": {"min_duration_off": params["segmentation"]["min_duration_off"]},
        "clustering": params["clustering"]
})

In [89]:
pipeline = SpeakerDiarization(
    segmentation=model,
    embedding=pipeline.embedding,
    embedding_exclude_overlap=pipeline.embedding_exclude_overlap,
    clustering=pipeline.klustering,
)

parameters_segm = parameters["params"]["segmentation"]
parameters_clust = parameters["params"]["clustering"]

pipeline.instantiate({
    "segmentation": {
        "min_duration_off": parameters_segm["min_duration_off"],
        "threshold": parameters_segm["threshold"]
    },
    "clustering": params["clustering"]
})

ValueError: parameter 'method' does not exist

In [47]:
pipeline.freeze({
    "segmentation": {
        "min_duration_off": parameters_segm["min_duration_off"],
        "threshold": parameters_segm["threshold"]
    },
    "clustering": {
        "method": parameters_clust["method"],
        "min_cluster_size": parameters_clust["min_cluster_size"],
        "threshold": parameters["threshold"]
    }
})

KeyError: 'threshold'

In [49]:
# TODO: niet hardgecodeerd voor eindresultaat
path = "../training_database/wavs/file17.wav"

In [50]:
# apply pretrained pipeline
diarization = pipeline(path)    

/home/lisa_verhoeyen/.local/lib/python3.10/site-packages/pyannote/audio/models/blocks/pooling.py:104: UserWarning: std(): degrees of freedom is <= 0. Correction should be strictly less than the reduction factor (input numel divided by output numel). (Triggered internally at /pytorch/aten/src/ATen/native/ReduceOps.cpp:1839.)
  std = sequences.std(dim=-1, correction=1)


create rttm file based on model result

In [51]:
file_name = re.search(r'.*\/(.+?)\.wav', path).group(1)

f = open(f"./results/rttm/{file_name}.rttm", "w")
for turn, _, speaker in diarization.itertracks(yield_label=True):
    f.write(f"SPEAKER {file_name} 1 {round(turn.start, 3)} {round(turn.end-turn.start, 3)} <NA> <NA> {speaker} <NA> <NA>\n")

f.close()

turn result into audio files

In [52]:
audio = AudioSegment.from_wav(f"../training_database/wavs/{file_name}.wav")
os.mkdir(f"./results/audio/{file_name}/")
for turn, _, speaker in diarization.itertracks(yield_label=True):
    if not os.path.isdir(f"./results/audio/{file_name}/{speaker}"):
        os.mkdir(f"./results/audio/{file_name}/{speaker}")

    segment = audio[turn.start * 1000:turn.end * 1000]  # Convert seconds to milliseconds
    segment.export(f"./results/audio/{file_name}/{speaker}/speaker_{speaker}_{turn.start:.1f}-{turn.end:.1f}.wav", format="wav")

create audio file based on speaker by combining audio files

In [53]:
speakers = os.listdir(f"./results/audio/{file_name}")

for speaker in speakers:
    files_list = os.listdir(f"./results/audio/{file_name}/{speaker}")

    combined = AudioSegment.from_file(f"./results/audio/{file_name}/{speaker}/{files_list[0]}")
    for i in range(1,len(files_list)):
        combined = combined + AudioSegment.from_file(f"./results/audio/{file_name}/{speaker}/{files_list[i]}")

    combined.export(f"./results/audio/{file_name}_{speaker}.wav", format="wav")